# Modelo Ligistic Regression

## Exploracion y limpieza de Datos

### Carga, Limpieza y Eliminación de Columnas

In [22]:
# Bloque 1 - Importamos las librerias necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
melb_df = pd.read_csv("Students_Data.csv")
melb_df.index += 1

##Eliminas las filas que tenga NaN (nulo)
melb_df = melb_df.dropna(axis=0)

columnas_a_eliminar = [
    'puntaje_interes_C',
    'puntaje_interes_H',
    'puntaje_interes_A',
    'puntaje_interes_S',
    'puntaje_interes_I',
    'puntaje_interes_D',
    'puntaje_interes_E',
    
    'puntaje_aptitud_C',
    'puntaje_aptitud_H',
    'puntaje_aptitud_A',
    'puntaje_aptitud_S',
    'puntaje_aptitud_I',
    'puntaje_aptitud_D',
    'puntaje_aptitud_E',

    'perfil_interes_1_codigo', 
    'perfil_interes_2_codigo', 
    'perfil_aptitud_1_codigo', 
    'perfil_aptitud_2_codigo'
]

# 2. Verifica cuáles de esas columnas existen realmente en tu DataFrame.
columnas_existentes = [col for col in columnas_a_eliminar if col in melb_df.columns]

# 3. Si se encontraron columnas, se eliminaran.
if columnas_existentes:
    melb_df.drop(columns=columnas_existentes, inplace=True, errors='ignore')
    print("Nuevas dimensiones del DataFrame:", melb_df.shape)
else:
    print("Las columnas de código especificadas ya no se encuentran en el DataFrame.")



Nuevas dimensiones del DataFrame: (217, 112)


### Definición de Objetivos (Y), Codificación y Separación X/Y

In [ ]:

df = melb_df.copy()

# Columnas objetivos a convertir
columnas_objetivo = [
    'perfil_interes_1_nombre', 'perfil_interes_2_nombre',
    'perfil_aptitud_1_nombre', 'perfil_aptitud_2_nombre',
    'carrera_interes_1', 'carrera_interes_2', 'carrera_interes_3',
    'carrera_interes_4', 'carrera_interes_5', 'carrera_aptitud_1',
    'carrera_aptitud_2', 'carrera_aptitud_3', 'carrera_aptitud_4',
    'carrera_aptitud_5'
]

# Filtra para asegurarte de que solo usas columnas que existen
columnas_existentes = [col for col in columnas_objetivo if col in df.columns]

print(f"Columnas encontradas para codificar: {len(columnas_existentes)}")

if columnas_existentes:
    todos_los_valores = pd.concat([df[col].astype(str) for col in columnas_existentes]).unique()
    le = LabelEncoder()
    le.fit(todos_los_valores)

    # 3. Aplica la transformación usando .loc para ser explícitos
    for col in columnas_existentes:
        # Usamos .loc[:, col] para indicar "todas las filas, esta columna"
        df.loc[:, col] = le.transform(df[col].astype(str))

else:
    print("¡Advertencia! Ninguna de las columnas objetivo especificadas se encontró.")


# Filtra solo las columnas que realmente existen
columnas_existentes = [col for col in columnas_objetivo if col in df.columns]

# 2. Separa X (Características) e Y (Objetivos)
Y = df[columnas_existentes]
X = df.drop(columns=columnas_existentes)

# 3. Divide los datos en Entrenamiento y Prueba (80/30)
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.20, random_state=42
)

# 4. Muestra las dimensiones de los conjuntos creados
print("--- Separación y División en Entrenamiento/Prueba Completada ---")
print(f"Forma de X_train: {X_train.shape}")
print(f"Forma de Y_train: {Y_train.shape}")
print(f"Forma de X_test:  {X_test.shape}")
print(f"Forma de Y_test:  {Y_test.shape}")


# 2. Convierte TODAS las columnas de Y_train e Y_test a tipo entero (int)
Y_train = Y_train.astype(int)
Y_test = Y_test.astype(int)
print(f"Todas las columnos son de tipo entero :)")

Columnas encontradas para codificar: 14
--- Separación y División en Entrenamiento/Prueba Completada ---
Forma de X_train: (173, 98)
Forma de Y_train: (173, 14)
Forma de X_test:  (44, 98)
Forma de Y_test:  (44, 14)


### Escalar los Datos (StandardScaler)

In [26]:
from sklearn.preprocessing import StandardScaler

print("Escalando los datos X...")

# 1. Crea la instancia del escalador
scaler = StandardScaler()

# 2. Ajusta el escalador CON X_train y transforma X_train
X_train_scaled = scaler.fit_transform(X_train)

# 3. Transforma X_test usando el escalador YA AJUSTADO
X_test_scaled = scaler.transform(X_test)

print("¡Datos X escalados y listos!")
print(f"Forma de X_train_scaled: {X_train_scaled.shape}")
print(f"Forma de X_test_scaled: {X_test_scaled.shape}")

Escalando los datos X...
¡Datos X escalados y listos!
Forma de X_train_scaled: (173, 98)
Forma de X_test_scaled: (44, 98)


### Entrenamiento del Modelo

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
import time

# --- Asumiendo que X_train_scaled y Y_train existen ---

print("\nIniciando el entrenamiento del modelo Logistic Regression (Multi-Output)...")

# 1. Crea el modelo base: Logistic Regression
#    - max_iter: Aumentamos las iteraciones para asegurar convergencia.
#    - random_state: Para reproducibilidad.
log_reg_base = LogisticRegression(max_iter=1000, random_state=42)

# 2. Crea el 'envoltorio' para múltiples salidas
modelo_lr = MultiOutputClassifier(log_reg_base, n_jobs=-1)

# 3. Entrena el modelo 'envuelto' con los datos escalados
start_time = time.time()
modelo_lr.fit(X_train_scaled, Y_train) # ¡¡OJO: Usa X_train_scaled!!
end_time = time.time()

print(f"¡Entrenamiento completado en {end_time - start_time:.2f} segundos!")

# --- Tu modelo 'modelo_lr' está entrenado ---


Iniciando el entrenamiento del modelo Logistic Regression (Multi-Output)...
¡Entrenamiento completado en 2.74 segundos!


### Predicciones

In [29]:
# --- Asumiendo que modelo_lr y X_test_scaled existen ---

print("\nRealizando predicciones con Logistic Regression...")

# Realiza predicciones sobre los datos de prueba ESCALADOS
Y_pred_lr = modelo_lr.predict(X_test_scaled) # ¡¡OJO: Usa X_test_scaled!!

print(f"¡Predicciones completadas! Se generaron {Y_pred_lr.shape[0]} predicciones.")


Realizando predicciones con Logistic Regression...
¡Predicciones completadas! Se generaron 44 predicciones.


### Validacion y Métricas

In [30]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
import pandas as pd

# --- Asumiendo que Y_test y Y_pred_lr existen ---

print("\n--- Evaluación del Modelo Logistic Regression ---")

# 1. Calcula la Precisión Exacta (Manual)
exact_matches_lr = np.all(Y_test.values == Y_pred_lr, axis=1).sum()
total_samples_lr = Y_test.shape[0]
exact_match_accuracy_lr = exact_matches_lr / total_samples_lr

# 2. Calcula las métricas para cada salida y sus promedios
f1_list_lr, precision_list_lr, recall_list_lr, accuracy_list_lr = [], [], [], []

print("\n--- Rendimiento Individual por Salida (Logistic Regression) ---")
for i, col_name in enumerate(Y_test.columns):
    y_true_col = Y_test.iloc[:, i]
    y_pred_col = Y_pred_lr[:, i] # ¡¡OJO: Usa Y_pred_lr!!
    
    acc = accuracy_score(y_true_col, y_pred_col)
    f1 = f1_score(y_true_col, y_pred_col, average='weighted', zero_division=0)
    prec = precision_score(y_true_col, y_pred_col, average='weighted', zero_division=0)
    rec = recall_score(y_true_col, y_pred_col, average='weighted', zero_division=0)
    
    accuracy_list_lr.append(acc)
    f1_list_lr.append(f1)
    precision_list_lr.append(prec)
    recall_list_lr.append(rec)
    
    print(f"  - {col_name}: Acc={acc:.3f}, F1={f1:.3f}, P={prec:.3f}, R={rec:.3f}")

# 3. Imprime los resultados generales
print("\n--- Métricas Generales (Logistic Regression) ---")
print(f"Precisión Exacta (Exact Match): {exact_match_accuracy_lr:.4f} ({exact_match_accuracy_lr * 100:.2f}%)")
print(f"Accuracy Promedio:              {np.mean(accuracy_list_lr):.4f}")
print(f"F1-Score Promedio (Weighted):   {np.mean(f1_list_lr):.4f}")
print(f"Precision Promedio (Weighted):  {np.mean(precision_list_lr):.4f}")
print(f"Recall Promedio (Weighted):     {np.mean(recall_list_lr):.4f}")
print("--------------------------------------------------")


--- Evaluación del Modelo Logistic Regression ---

--- Rendimiento Individual por Salida (Logistic Regression) ---
  - perfil_interes_1_nombre: Acc=0.659, F1=0.639, P=0.632, R=0.659
  - perfil_interes_2_nombre: Acc=0.295, F1=0.283, P=0.304, R=0.295
  - perfil_aptitud_1_nombre: Acc=0.682, F1=0.651, P=0.636, R=0.682
  - perfil_aptitud_2_nombre: Acc=0.341, F1=0.329, P=0.326, R=0.341
  - carrera_interes_1: Acc=0.500, F1=0.446, P=0.472, R=0.500
  - carrera_interes_2: Acc=0.477, F1=0.450, P=0.435, R=0.477
  - carrera_interes_3: Acc=0.545, F1=0.519, P=0.602, R=0.545
  - carrera_interes_4: Acc=0.477, F1=0.463, P=0.544, R=0.477
  - carrera_interes_5: Acc=0.455, F1=0.432, P=0.518, R=0.455
  - carrera_aptitud_1: Acc=0.500, F1=0.440, P=0.430, R=0.500
  - carrera_aptitud_2: Acc=0.500, F1=0.430, P=0.392, R=0.500
  - carrera_aptitud_3: Acc=0.523, F1=0.468, P=0.457, R=0.523
  - carrera_aptitud_4: Acc=0.523, F1=0.479, P=0.486, R=0.523
  - carrera_aptitud_5: Acc=0.568, F1=0.520, P=0.514, R=0.568

--- M